# 01_etl_stage_to_bronze
---
Este notebook executa o processo `ETL` que transfere os dados da camada **Stage** para a **Bronze**. Englobando verificação, padronização, conversão e movimentação dos arquivos necessários para a continuidade do pipeline.


In [5]:
# Parameters

stage_path = "/opt/airflow/data-layer/stage"
bronze_path = "/opt/airflow/data-layer/bronze"


In [6]:
import os
from datetime import datetime
from pathlib import Path

from pyspark.sql import DataFrame, SparkSession

from transformer.utils.file_io import (check_files_in_folder, delete_files, move_files)
from transformer.utils.logger import get_logger
from transformer.utils.quality_gates_bronze import run_quality_gates_bronze
from transformer.utils.spark_helpers import get_spark_session


## Job 0: kaggle_download_and_prepare

Este job executa o download, a descompactação e a preparação dos arquivos para a camada Stage. Sua execução permanece desativada porque depende de configurações manuais de acesso à API do Kaggle. Reative as células e informe suas credenciais caso deseje executá-lo.


In [7]:
%%script false --no-raise-error # Comentar essa linha se quiser rodar a célula.
import numpy as np
import pandas as pd

log = get_logger("kaggle_and_prepare")

NUM_CHUNKS=10


### Definindo funções de download e de divisão

In [8]:
%%script false --no-raise-error # Comentar essa linha se quiser rodar a célula.
def download_kaggle_dataset(dataset_name: str, dest_path: Path) -> None:
    """
    Faz o download e a extração de um dataset do Kaggle, caso ainda não exista localmente.

    Args:
        dataset_name (str): Nome do dataset no Kaggle (ex: 'usdot/flight-delays').
        dest_path (Path): Caminho destino para os arquivos extraídos.

    Raises:
        Exception: Para erros de autenticação ou rede.
    """
    try:
        existing_files = os.listdir(dest_path)
        has_csv_files = any(file.endswith(".csv") for file in existing_files)

        if has_csv_files:
            log.info(f"[Kaggle] Arquivos já existem em '{dest_path}'. Pulando download.")
            return

        log.info(f"[Kaggle] Nenhum arquivo CSV encontrado. Baixando o dataset '{dataset_name}'.")
        kaggle.api.dataset_download_files(dataset_name, path=dest_path, unzip=True)
        log.info("[Kaggle] Download e extração concluídos com sucesso.")
    except Exception as e:
        log.exception(f"[Kaggle] Ocorreu um erro! Verifique se a API do Kaggle está configurada: {e}.")
        log.exception(f"[Kaggle] Erro: {e}.")
        raise

def split_main_file(source_file: Path, num_chunks: int = NUM_CHUNKS) -> None:
    """
    Divide o arquivo flights.csv em múltiplos chunks de tamanho aproximadamente igual.

    Args:
        source_file (Path): Caminho do arquivo csv principal.
        num_chunks (int): Número de partes a gerar.

    Raises:
        FileNotFoundError: Se o arquivo não existir.
        ValueError: Se o arquivo estiver vazio.
        Exception: Para erros inesperados durante a divisão.
    """
    if not source_file.exists():
        raise FileNotFoundError(f"[Landing] Arquivo não encontrado: '{source_file}'.")

    try:
        df = pd.read_csv(source_file)
        if df.empty:
            raise ValueError(f"O arquivo '{source_file}' está vazio.")

        log.info(f"[Split] Dividindo '{source_file.name}' em {num_chunks} partes aproximadamente iguais.")

        chunks = np.array_split(df, num_chunks)

        for i, chunk_df in enumerate(chunks, start=1):
            chunk_name = f"{source_file.stem}_part_{i:02d}{source_file.suffix}"
            chunk_path = source_file.parent / chunk_name
            chunk_df.to_csv(chunk_path, index=False)
            log.info(f"[Split] Chunk salvo: '{chunk_name}', {len(chunk_df)} linhas.")

        log.info("[Split] Divisão do arquivo concluída com sucesso.")

    except Exception as e:
        log.error(f"[Split] Erro ao dividir o arquivo: {e}.")
        raise


### Runner para o job `kaggle_download_and_prepare`

In [9]:
%%script false --no-raise-error # Comentar essa linha se quiser rodar a célula.
try:
    # Download
    download_kaggle_dataset(DATASET_NAME, stage_path)

    # Divisão
    flights_file = Path(stage_path) / "flights.csv"
    if flights_file.exists():
        split_main_file(flights_file, num_chunks=NUM_CHUNKS)
    else:
        log.warning(f"[Landing] Arquivo 'flights.csv' não encontrado. Verifique a pasta.")

except Exception as e:
    log.error(f"[Landing] Falha no processo de preparação: {e}")


## Job 1: unify_flight_chunks

Este job realiza a unificação dos arquivos `flights_part_*.csv` presentes na camada **Stage**, consolidando-os em um único arquivo Parquet `flights.parquet`.

In [10]:
log = get_logger("unify_chunks")

spark = get_spark_session("UnifyFlightChunks")
log.info("[UnifyChunks] SparkSession iniciada.")

# Ajustes de performance para o Spark
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.shuffle.partitions", "32")


2025-11-16 02:58:51 [INFO] unify_chunks | [INFO] Logger inicializado no modo standalone (INFO).
/usr/local/lib/python3.12/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found


:: loading settings :: url = jar:file:/usr/local/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6d17fbee-f44d-4702-9909-9a8a893c470a;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.3 in central
	found org.checkerframework#checker-qual;3.42.0 in central
downloading https://repo1.maven.org/maven2/org/postgresql/postgresql/42.7.3/postgresql-42.7.3.jar ...
	[SUCCESSFUL ] org.postgresql#postgresql;42.7.3!postgresql.jar (324ms)
downloading https://repo1.maven.org/maven2/org/checkerframework/checker-qual/3.42.0/checker-qual-3.42.0.jar ...
	[SUCCESSFUL ] org.checkerframework#checker-qual;3.42.0!checker-qual.jar (118ms)
:: resolution report :: resolve 4088ms :: artifacts dl 448ms
	:: modules in use:
	org.checkerframework#checker-qual;3.42.0 from central in [default]
	org.postgresql#postgresql;42.7.3 from central in [default]
	-------------------------------------------

### Definindo função de unificação

In [11]:
def reassemble_chunks(spark: SparkSession, chunk_files: list[str], header: bool = True) -> DataFrame:
    """
    Lê múltiplos arquivos de chunk (flights_part_*.csv) e os unifica em um único DataFrame Spark.

    Args:
        spark (SparkSession): Sessão Spark ativa.
        chunk_files (list[str]): Lista de caminhos completos dos arquivos csv a serem unificados.
        header (bool, opcional): Define se os arquivos csv possuem cabeçalho. Padrão: True.

    Returns:
        DataFrame: DataFrame Spark consolidado com todos os chunks.
    """
    if not chunk_files:
        raise ValueError("[UnifyChunks][ERROR] Nenhum arquivo de chunk fornecido para unificação.")

    log.info(f"[UnifyChunks] Lendo e concatenando {len(chunk_files)} arquivo(s) de chunk.")

    try:
        df = (
            spark.read
            .option("header", header)
            .option("inferSchema", False)
            .csv(chunk_files)
        )
        
        log.info("[UnifyChunks] Unificação dos chunks concluída com sucesso.")
        
        return df

    except Exception as e:
        log.error(f"[UnifyChunks][ERROR] Falha ao ler os arquivos csv: {e}.")
        
        raise


### Runner para o job `unify_flight_chunks`

In [12]:
try:
    log.info("[UnifyChunks] Iniciando job de unificação de chunks.")

    # Localiza arquivos csv na stage
    csv_files = check_files_in_folder(stage_path, "*.csv")
    chunk_files = [f for f in csv_files if "flights_part" in f]

    if not chunk_files:
        raise FileNotFoundError(f"[UnifyChunks][ERROR] Nenhum arquivo de chunk encontrado em {stage_path}.")

    # Unifica os chunks
    df_unified = reassemble_chunks(spark, chunk_files)

    # Executa quality gates
    required_columns = [
        'YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 
        'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 
        'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 
        'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 
        'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 
        'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'
    ]
    run_quality_gates_bronze(df_unified, "flights_bronze", required_columns)

    # Salva o arquivo unificado
    stage_output = f"{stage_path}/flights.parquet"
    df_unified.write.mode("overwrite").option("compression", "snappy").parquet(stage_output)
    
    log.info(f"[UnifyChunks] Arquivo unificado salvo em: {stage_output}.")

except Exception as e:
    log.exception(f"[UnifyChunks][ERROR] Falha durante execução: {e}")
    raise

finally:
    log.info("[UnifyChunks] Job de unificação de chunks encerrado.")


2025-11-16 02:59:06 [INFO] unify_chunks | [UnifyChunks] Iniciando job de unificação de chunks.
2025-11-16 02:59:06 [INFO] file_io | [INFO] Encontrados 12 arquivo(s).
2025-11-16 02:59:06 [INFO] unify_chunks | [UnifyChunks] Lendo e concatenando 10 arquivo(s) de chunk.
2025-11-16 02:59:09 [INFO] unify_chunks | [UnifyChunks] Unificação dos chunks concluída com sucesso.
2025-11-16 02:59:09 [INFO] quality_gates_bronze | [Quality][Landing] Iniciando validações do dataset 'flights_bronze'.
2025-11-16 02:59:10 [INFO] quality_gates_bronze | [Quality][Landing]       _check_row_count_not_empty: OK
2025-11-16 02:59:10 [INFO] quality_gates_bronze | [Quality][Landing]       _check_schema_columns: OK
2025-11-16 02:59:10 [INFO] quality_gates_bronze | [Quality][Landing] Todas as validações para 'flights_bronze' concluídas com sucesso.
25/11/16 02:59:11 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
2025-11-16 02:59

In [13]:
# Encerra a sessão Spark para o job
spark.stop()
log.info("[UnifyChunks] Sessão Spark finalizada.")


2025-11-16 03:00:06 [INFO] unify_chunks | [UnifyChunks] Sessão Spark finalizada.


## Job 2: convert_csv_to_parquet

Este job realiza a conversão dos arquivos csv presentes na camada **Stage** (exceto os chunks `flights_part_*`) para o formato **Parquet**, mantendo o schema original dos dados.


In [14]:
log = get_logger("convert_csv")

spark = get_spark_session("ConvertCsvToParquet")
log.info("[ConvertCSV] SparkSession iniciada.")

# Ajustes de performance para o Spark
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.shuffle.partitions", "32")


2025-11-16 03:02:08 [INFO] convert_csv | [INFO] Logger inicializado no modo standalone (INFO).
2025-11-16 03:02:08 [INFO] spark_helpers | [INFO] SparkSession criada com sucesso: 'ConvertCsvToParquet' (master=local[*]).
2025-11-16 03:02:08 [INFO] convert_csv | [ConvertCSV] SparkSession iniciada.


### Definindo função de conversão

In [15]:
def convert_csv_to_parquet(spark: SparkSession, csv_files: list[str], stage_path: str) -> None:
    """
    Converte arquivos csv em formato parquet.

    Args:
        spark (SparkSession): Sessão Spark ativa.
        csv_files (list[str]): Lista de caminhos dos arquivos csv a converter.
        stage_path (str): Caminho base da camada Stage.
    """
    if not csv_files:
        raise ValueError("[ConvertCSV][ERROR] Nenhum arquivo csv fornecido para conversão.")

    for csv in csv_files:
        try:
            # Resolve caminhho do arquivo
            base_name = os.path.basename(csv).replace(".csv", ".parquet")

            log.info(f"[ConvertCSV] Lendo arquivo csv: {csv}.")

            df = (
                spark.read
                .option("header", True)
                .option("inferSchema", False)
                .csv(csv)
            )

            # Executa quality gates
            if df.columns[1] == 'AIRLINE':
                required_columns = ['IATA_CODE', 'AIRLINE']
            else: 
                required_columns = ['IATA_CODE', 'AIRPORT', 'CITY', 'STATE', 'COUNTRY', 'LATITUDE', 'LONGITUDE']
            
            run_quality_gates_bronze(df, base_name, required_columns)

            # Converte em parquet
            parquet_path = f"{stage_path}/{base_name}"
            df.write.mode("overwrite").option("compression", "snappy").parquet(parquet_path)
            
            log.info(f"[ConvertCSV] Arquivo convertido: {parquet_path}.")

        except Exception as e:
            log.error(f"[ConvertCSV][ERROR] Falha ao converter {csv}: {e}.")
            raise IOError(f"Erro ao processar {csv}: {e}.") from e


### Runner para o job `convert_csv_to_parquet`

In [16]:
try:
    log.info("[ConvertCSV] Iniciando job de conversão de csv para parquet.")

    csv_files = check_files_in_folder(stage_path, "*.csv")
    target_files = [f for f in csv_files if "flights_part" not in f]

    if not target_files:
        raise FileNotFoundError(f"[Landing][ConvertCSV][ERROR] Nenhum arquivo csv com o padrão encontrado em {stage_path}.")

    convert_csv_to_parquet(spark, target_files, stage_path)

    log.info(f"[ConvertCSV] Conversão concluída. {len(target_files)} arquivo(s) processado(s).")

except Exception as e:
    log.exception(f"[ConvertCSV][ERROR] Falha durante execução: {e}.")
    raise

finally:
    log.info("[ConvertCSV] Job de conversão encerrado.")


2025-11-16 03:02:15 [INFO] convert_csv | [ConvertCSV] Iniciando job de conversão de csv para parquet.
2025-11-16 03:02:15 [INFO] file_io | [INFO] Encontrados 12 arquivo(s).
2025-11-16 03:02:15 [INFO] convert_csv | [ConvertCSV] Lendo arquivo csv: /opt/airflow/data-layer/stage/airlines.csv.
2025-11-16 03:02:15 [INFO] quality_gates_bronze | [Quality][Landing] Iniciando validações do dataset 'airlines.parquet'.
2025-11-16 03:02:16 [INFO] quality_gates_bronze | [Quality][Landing]       _check_row_count_not_empty: OK
2025-11-16 03:02:16 [INFO] quality_gates_bronze | [Quality][Landing]       _check_schema_columns: OK
2025-11-16 03:02:16 [INFO] quality_gates_bronze | [Quality][Landing] Todas as validações para 'airlines.parquet' concluídas com sucesso.
2025-11-16 03:02:16 [INFO] convert_csv | [ConvertCSV] Arquivo convertido: /opt/airflow/data-layer/stage/airlines.parquet.
2025-11-16 03:02:16 [INFO] convert_csv | [ConvertCSV] Lendo arquivo csv: /opt/airflow/data-layer/stage/airports.csv.
2025-1

In [17]:
# Encerra a sessão Spark para o job
spark.stop()
log.info("[ConvertCSV] Sessão Spark finalizada.")


2025-11-16 03:02:25 [INFO] convert_csv | [ConvertCSV] Sessão Spark finalizada.


## Job 3: move_files_to_bronze

Este job move os arquivos `parquet` da camada **Stage** para a camada **Bronze**, organizando-os por data de processamento.


In [18]:
log = get_logger("move_to_bronze")

spark = get_spark_session("MoveStageToBronze")
log.info("[MoveToBronze] SparkSession iniciada.")


2025-11-16 03:02:36 [INFO] move_to_bronze | [INFO] Logger inicializado no modo standalone (INFO).
2025-11-16 03:02:36 [INFO] spark_helpers | [INFO] SparkSession criada com sucesso: 'MoveStageToBronze' (master=local[*]).
2025-11-16 03:02:36 [INFO] move_to_bronze | [MoveToBronze] SparkSession iniciada.


### Runner para o job `move_files_to_bronze`

In [19]:
try:
    log.info("[MoveToBronze] Iniciando job de movimentação de arquivos.")

    parquet_files = check_files_in_folder(stage_path, "*.parquet")
    if not parquet_files:
        raise FileNotFoundError(f"[MoveToBronze][ERROR] Nenhum arquivo Parquet encontrado em {stage_path}.")

    processing_date = datetime.now().strftime("%Y-%m-%d")

    move_files(
        spark=spark,
        source_files=parquet_files,
        base_dest_path=bronze_path,
        processing_date=processing_date,
    )

    log.info(f"[MoveToBronze] {len(parquet_files)} arquivo(s) movido(s) para bronze/{processing_date}.")

except Exception as e:
    log.exception(f"[Landing][MoveToBronze][ERROR] Falha durante execução: {e}")
    raise

finally:
    log.info("[MoveToBronze] Job de movimentação encerrado.")


2025-11-16 03:02:40 [INFO] move_to_bronze | [MoveToBronze] Iniciando job de movimentação de arquivos.
2025-11-16 03:02:40 [INFO] file_io | [INFO] Encontrados 3 arquivo(s).
2025-11-16 03:02:40 [INFO] file_io | [INFO] Movendo arquivos para '/opt/airflow/data-layer/bronze'.
2025-11-16 03:02:40 [INFO] file_io | [INFO] Diretório criado: /opt/airflow/data-layer/bronze/2025-11-16/PARQUET
2025-11-16 03:02:40 [INFO] file_io | [INFO] 'airlines.parquet' movido para '/opt/airflow/data-layer/bronze/2025-11-16/PARQUET/airlines.parquet'.
2025-11-16 03:02:40 [INFO] file_io | [INFO] 'airports.parquet' movido para '/opt/airflow/data-layer/bronze/2025-11-16/PARQUET/airports.parquet'.
2025-11-16 03:02:40 [INFO] file_io | [INFO] 'flights.parquet' movido para '/opt/airflow/data-layer/bronze/2025-11-16/PARQUET/flights.parquet'.
2025-11-16 03:02:40 [INFO] file_io | [INFO] Movimentação concluída com sucesso.
2025-11-16 03:02:40 [INFO] move_to_bronze | [MoveToBronze] 3 arquivo(s) movido(s) para bronze/2025-11-1

In [20]:
# Encerra a sessão Spark para o job
spark.stop()
log.info("[MoveToBronze] Sessão Spark finalizada.")


2025-11-16 03:02:46 [INFO] move_to_bronze | [MoveToBronze] Sessão Spark finalizada.


## Job 4: cleanup_stage

Este job remove os arquivos `csv` e `parquet` da camada **Stage** após a conclusão do carregamento na camada **Bronze**.


In [21]:
log = get_logger("cleanup_stage")

spark = get_spark_session("CleanupStage")
log.info("[CleanupStage] SparkSession iniciada.")


2025-11-16 03:02:54 [INFO] cleanup_stage | [INFO] Logger inicializado no modo standalone (INFO).
2025-11-16 03:02:54 [INFO] spark_helpers | [INFO] SparkSession criada com sucesso: 'CleanupStage' (master=local[*]).
2025-11-16 03:02:54 [INFO] cleanup_stage | [CleanupStage] SparkSession iniciada.


### Runner para o job `cleanup_stage`

In [22]:
try:
    log.info("[CleanupStage] Iniciando job de limpeza da stage.")

    removable_files = check_files_in_folder(stage_path, "*.*")
    if not removable_files:
        log.warning("[CleanupStage] Nenhum arquivo encontrado para remoção.")
    else:
        delete_files(spark, removable_files)
        log.info(f"[CleanupStage] {len(removable_files)} arquivo(s) removido(s) da stage.")

except Exception as e:
    log.exception(f"[CleanupStage][ERROR] Falha durante execução: {e}.")
    raise

finally:
    log.info("[CleanupStage] Job de limpeza encerrado.")


2025-11-16 03:02:57 [INFO] cleanup_stage | [CleanupStage] Iniciando job de limpeza da stage.
2025-11-16 03:02:57 [INFO] file_io | [INFO] Encontrados 12 arquivo(s).
2025-11-16 03:02:57 [INFO] file_io | [INFO] Deletando 12 arquivo(s).
2025-11-16 03:02:57 [INFO] file_io | [INFO] '/opt/airflow/data-layer/stage/airlines.csv' deletado com sucesso.
2025-11-16 03:02:58 [INFO] file_io | [INFO] '/opt/airflow/data-layer/stage/airports.csv' deletado com sucesso.
2025-11-16 03:02:58 [INFO] file_io | [INFO] '/opt/airflow/data-layer/stage/flights_part_01.csv' deletado com sucesso.
2025-11-16 03:02:58 [INFO] file_io | [INFO] '/opt/airflow/data-layer/stage/flights_part_02.csv' deletado com sucesso.
2025-11-16 03:02:58 [INFO] file_io | [INFO] '/opt/airflow/data-layer/stage/flights_part_03.csv' deletado com sucesso.
2025-11-16 03:02:58 [INFO] file_io | [INFO] '/opt/airflow/data-layer/stage/flights_part_04.csv' deletado com sucesso.
2025-11-16 03:02:58 [INFO] file_io | [INFO] '/opt/airflow/data-layer/stag

In [23]:
# Encerra a sessão Spark para o job
spark.stop()
log.info("[CleanupStage] Sessão Spark finalizada.")


2025-11-16 03:03:05 [INFO] cleanup_stage | [CleanupStage] Sessão Spark finalizada.
